In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import label_ranking_average_precision_score

In [2]:
NUM_FEATURES = 5000
NUM_CLASSES = 3993

In [3]:
#Get true predictions
y_val = pd.read_csv("../data/expanded/dev_labels.csv", names=range(NUM_CLASSES))

In [4]:
#Tree
y_rf = pd.read_csv("../public_data/saved_rf_probabilities.csv", names=range(NUM_CLASSES),header= 0)

In [5]:
score_rf = label_ranking_average_precision_score(y_val,y_rf)
score_rf

0.5604580391775857

In [215]:
#KNN
y_knn = pd.read_csv("../public_data/knn.csv", names=range(NUM_CLASSES),header = 0)

In [216]:
score_knn = label_ranking_average_precision_score(y_val,y_knn)
score_knn

0.2912844932011161

In [7]:
#SVM
y_svm = pd.read_csv("../public_data/svm.csv", names=range(NUM_CLASSES))

In [8]:
score_svm = label_ranking_average_precision_score(y_val,y_svm)
score_svm

0.3391564618701288

In [118]:
#NN
from numpy import load
y_nn = np.load('../public_data/nn_predictions.npz')
y_nn = y_nn['predictions']

In [119]:
score_nn = label_ranking_average_precision_score(y_val,y_nn)
score_nn

0.6254337395360583

# Convex combination

In [ ]:
#Do a convex combination based upon validation scores
#Note convex combination does not seem to work the best.
total = score_rf + score_knn + score_svm + score_mlp

rf_w = score_rf / total
knn_w = score_knn / total
svm_w = score_svm/total
mlp_w = score_nn/total

In [207]:
#Can use convex combination or set manually.
#ensemble = 0.8*y_rf.values + 0.19*y_svm.values +0.01*y_knn.values
ensemble = 0.85*y_nn + 0.127*y_rf.values + 0.023*y_svm.values + 0*y_knn.values

In [208]:
score_ensemble = label_ranking_average_precision_score(y_val,ensemble)

In [209]:
#Small improvement over sole rf
score_ensemble

0.6345250822842396

# Ensemble weight tuning 

# All 4 : NN, RF, SVM and KNN

In [217]:
import random

In [222]:
i = 0
best_score = 0
while i< 200:
    
    if i%10 ==0:
        print(i)
        
    y_test_nn = np.random.uniform(0.7,1)
    y_test_rf = np.random.uniform(0,1-y_test_nn)
    y_test_svm = np.random.uniform(0,1-y_test_nn-y_test_rf)
    y_test_knn = 1-y_test_nn-y_test_rf-y_test_svm
    
    
    ensemble_score = y_test_nn*y_nn + y_test_rf*y_rf.values + y_test_svm*y_svm.values +y_test_knn*y_knn.values
    temp_score = label_ranking_average_precision_score(y_val,ensemble_score)
    
    if temp_score > best_score:
        best_score = temp_score
        
        best_weights = []
        best_weights.append(y_test_nn)
        best_weights.append(y_test_rf)
        best_weights.append(y_test_svm)
        best_weights.append(y_test_knn)
    i+=1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [224]:
best_weights

[0.7484868940173159,
 0.12187404054186865,
 0.06921964707939315,
 0.060419418361422256]

In [223]:
best_score

0.6351116290635036

# NN and RF

In [188]:
i = 0
best_score = 0
while i< 100:
    
    if i%10 ==0:
        print(i)
        
    y_test_nn = np.random.uniform(0.4,1)
    y_test_rf = 1-y_test_nn
    
    
    ensemble_score = y_test_nn*y_nn + y_test_rf*y_rf.values 
    temp_score = label_ranking_average_precision_score(y_val,ensemble_score)
    
    if temp_score > best_score:
        best_score = temp_score
        
        best_weights = []
        best_weights.append(y_test_nn)
        best_weights.append(y_test_rf)
    i+=1

0
10
20
30
40
50
60
70
80
90


In [189]:
best_score

0.6290318995802494

# NN, RF and SVM (best combination)

In [201]:
i = 0
best_score = 0
while i< 500:
    
    if i%10 ==0:
        print(i)
        
    y_test_nn = np.random.uniform(0.4,1)
    y_test_rf = np.random.uniform(0,1-y_test_nn)
    y_test_svm = 1-y_test_nn-y_test_rf    
    
    ensemble_score = y_test_nn*y_nn + y_test_rf*y_rf.values + y_test_svm*y_svm.values
    temp_score = label_ranking_average_precision_score(y_val,ensemble_score)
    
    if temp_score > best_score:
        best_score = temp_score
        
        best_weights = []
        best_weights.append(y_test_nn)
        best_weights.append(y_test_rf)
        best_weights.append(y_test_svm)
    i+=1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [202]:
best_score

0.6345056739829807

In [203]:
best_weights

[0.8437156989129617, 0.13313503152459621, 0.023149269562442065]

# NN and SVM

In [196]:
i = 0
best_score = 0
while i< 100:
    
    if i%10 ==0:
        print(i)
        
    y_test_nn = np.random.uniform(0.4,1)
    y_test_svm = 1-y_test_nn  
    
    ensemble_score = y_test_nn*y_nn + y_test_svm*y_svm.values
    temp_score = label_ranking_average_precision_score(y_val,ensemble_score)
    
    if temp_score > best_score:
        best_score = temp_score
        
        best_weights = []
        best_weights.append(y_test_nn)
        best_weights.append(y_test_svm)
    i+=1

0
10
20
30
40
50
60
70
80
90


In [197]:
best_score

0.6308784881574803